In [9]:
pip install flask-cors

  Using cached flask_cors-6.0.1-py3-none-any.whl.metadata (5.3 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import random
import plotly.graph_objects as go
from matplotlib.gridspec import GridSpec
import plotly.figure_factory as ff
import dash
from dash import html, dcc
import plotly.express as px
import random
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import flask
import plotly.express as px
from flask_cors import CORS

import warnings  

In [4]:
data = pd.read_csv("C:\\Users\\andre\\Desktop\\IMUproject\\global_food_wastage_dataset.csv")

In [ ]:
data.dtypes

In [13]:
metric_map = {
    "total_waste_(tons)": "Total Waste (Tons)",
    "economic_loss_(million_$)": "Economic Loss (Million $)",
    "avg_waste_per_capita_(kg)": "Avg Waste per Capita (Kg)",
    "household_waste_(%)": "Household Waste (%)"
}

app = dash.Dash(__name__)
app.title = "Global Food Wastage Dashboard"

app.layout = html.Div([
    html.H1("Global Food Wastage", style={
        'textAlign': 'center',
        'color': 'dark green',
        'fontFamily': 'Fraunces, serif',
        'fontWeight': 'bold'
    }),

    html.Label("Filter by Country:"),
    dcc.Dropdown(
        id="country_filter",
        options=[{"label": c, "value": c} for c in sorted(data["Country"].unique())],
        multi=True,
        placeholder="Select Country(s)"
    ),

    html.Label("Filter by Year:"),
    dcc.Dropdown(
        id="year_filter",
        options=[{"label": y, "value": y} for y in sorted(data["Year"].unique())],
        multi=False,
        placeholder="Select Year"
    ),

    html.Label("Filter by Food Category(s):"),
    dcc.Dropdown(
        id="food_category_filter",
        options=[{"label": f, "value": f} for f in sorted(data["Food Category"].unique())],
        multi=True,
        placeholder="Select Food Category(s)"
    ),

    html.Label("Select Metric:"),
    dcc.Dropdown(
        id="metric_filter",
        options=[
            {"label": "Total Waste (Tons)", "value": "total_waste_(tons)"},
            {"label": "Economic Loss (Million $)", "value": "economic_loss_(million_$)"},
            {"label": "Avg Waste per Capita (Kg)", "value": "avg_waste_per_capita_(kg)"},
            {"label": "Household Waste (%)", "value": "household_waste_(%)"}
        ],
        value="total_waste_(tons)",
        clearable=False
    ),

    dcc.Graph(id="chart")
], style={"padding": "20px"})



@app.callback(
    Output("chart", "figure"),
    Input("country_filter", "value"),
    Input("year_filter", "value"),
    Input("food_category_filter", "value"),
    Input("metric_filter", "value")
)
def update_chart(selected_country, selected_year, selected_food_category, selected_metric):
    filtered_data = data.copy()

    if selected_country:
        filtered_data = filtered_data[filtered_data["Country"].isin(selected_country)]
    if selected_year:
        filtered_data = filtered_data[filtered_data["Year"] == selected_year]
    if selected_food_category:
        filtered_data = filtered_data[filtered_data["Food Category"].isin(selected_food_category)]


    metric_column = metric_map[selected_metric]

    if filtered_data.empty:
        return px.bar(title="No data available for selected filters")

    fig = px.bar(
        filtered_data,
        x="Food Category",
        y=metric_column,
        color="Country",
        barmode="group",
        labels={metric_column: metric_column}
    )

    fig.update_layout(
        title=f"{metric_column} by Food Category ({selected_year})",
        xaxis_title="Food Category",
        yaxis_title=metric_column,
        legend_title="Country"
    )

    return fig

if __name__ == "__main__":
    random_port = random.randint(8050, 9000)
    app.run(debug=True, port=random_port)